# Summarization Frame Work.

### Dowloading and Saving the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
save_directory = "./bart-large-cnn"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save the tokenizer and model locally
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

### Summarization

In [1]:
import torch
import numpy as np
import pandas as pd
import sklearn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
save_directory = "./bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [3]:

def summarizer(text, max_length=150, min_length=40):
    
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [4]:
src_text = """
OpenAI’s GPT-3 has been making waves in the AI community. With 175 billion parameters, it is the largest language model ever created. 
It has applications in various domains including translation, summarization, and even creative writing. 
GPT-3 can generate human-like text, making it a powerful tool for natural language understanding and generation. 
However, the model also raises ethical concerns regarding its potential misuse and the environmental impact of training such large models.
"""

In [5]:
summarized_text = summarizer(src_text)
print(summarized_text)

With 175 billion parameters, it is the largest language model ever created. It has applications in various domains including translation, summarization, and even creative writing. The model also raises ethical concerns regarding its potential misuse and the environmental impact of training such large models.


### For long texts

In [ ]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import math
from transformers import BartTokenizer, BartForConditionalGeneration

# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Define the path to the model
model_id = "facebook/bart-large-cnn"
local_model_path = model_id

# Load the tokenizer and model from the local directory
tokenizer = BartTokenizer.from_pretrained(local_model_path)
model = BartForConditionalGeneration.from_pretrained(local_model_path).to(device)

def summarize_text(text, max_length=150, min_length=40):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text, return_tensors="pt")[0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk))
    return chunks

def iterative_summarization(text, target_word_count=100):
    while True:
        chunks = chunk_text(text)
        summaries = [summarize_text(chunk) for chunk in chunks]
        combined_summary = " ".join(summaries)
        
        word_count = len(combined_summary.split())
        if word_count <= target_word_count:
            return combined_summary
        
        text = combined_summary


In [ ]:
# Sample text to summarize
text = """
There's a hot new club in town that everyone is itching to gain entrance into. The entry is free, and there are no membership fees or exclusive conditions. Literally all you have to do, is wake up and show up. The majority of us already know the importance of starting our day right, but getting up at 5 am might sound a little too "alarming." However, if you want to maximize your productivity, and show up with the right energy and mindset for what matters, then you might want to join the 5 am Club. The 5 am Club, is a self-help parable that shows us how to embrace a revolutionary morning routine, that delivers results. World-famous productivity and leadership expert, Robin Sharma teaches us how to use the first hour of our day to harness our creative capacity, protect our sanity, and drive personal growth. Author Robin Sharma believes that when we rise at 5 am, when the world is quiet and devoid of energy-sapping distractions, this is when we learn to master ourselves. The argument is that an early rise results in our creative and productive potential being harvested, which allows us to stay grounded and focused for the day ahead. We'll briefly explore why an early morning routine can help boost creativity and productivity. Furthermore, a productive morning routine is incomplete without a dose of inward reflection, so we'll examine what Sharma calls "the four interior empires." The four interior empires are, mindset, heartset, healthset and soulset, and each should be contemplated in order to lead a balanced life. We'll then unpack some of the ways that can help us structure the perfect morning routine. This willensure that there's a payoff to getting up at 5 am, which delivers results and is worth ignoring the snooze button for. What's in It for Me? Waking up at 5 in the morning boosts creativity, energy, and productivity. How often do you sit down in the morning to start work, only to get caught up doing tasks that seem urgent, but are actually unimportant? It's easy to spend "work time" replying to emails and calls, responding to social media messages, and scrolling through the latest news feed. We flip our focus from one task to another, and by the time we reach lunchtime, we're tired from our morning's work, but without much to show for our "efforts." The onslaught of daily distractions steals our mental energy and depletes the cognitive resources required for optimal productivity. However, at 5 am when the world sleeps, we have the golden opportunity to wake up and channel our focus and creative energy towards accomplishing goals that really matter. A more peaceful start to the morning also stimulates the production of two mood-boosting neurotransmitters: dopamine and serotonin. What's more, during this early tranquil hour, our brains are at their best. Early in the morning the brain's prefrontal cortex (responsible for rational thought) temporarily shuts down. The scientific term for this is "transient hypofrontality." All it means is that for a while, under certain conditions, various brain regions rest, while other areas become more dominant. More specifically, frontal regions that light up when we over-analyze and stress, appear to be dormant at this hour. On the other hand, areasresponsible for more creative flow-like thinking, switch on. So, if you want to get into "the zone," start an early alarm, because this is the time when our brain is more likely to enter a state of flow-like thinking. Starting each morning early, allows us to cash in on that feel-good mood. This is the time when we're most undistracted and engaged in flow-type thought, which helps us to feel inspired, energized, and "in the zone." The trick is to use this highly productive time wisely. The 5 am Club doesn't just require that we're up by 5 am; it means we dedicate our early hours to deep work and high-value activities. The great composer Wolfgang Amadeus Mozart, was an early bird who used the quiet hours to practice and refine his musical craft. Novelist John Grisham also attributes his success to his early morning writing routine. Sharma says, 'This habit should not be underestimated, being an early riser is one of the critical success factors that most great people in the world practice daily.' No matter our craft, mastery takes practice. If we use our morning hours wisely, and dedicate focused, undisturbed time to hone our skills, who knows what we'll achieve? Sharma reminds us that just 1% improvement every day for 30 days, elevates our performance by 30%. What we do daily is more important than what we do every so often. Another crucial aspect of optimizing our early hours, is to dedicate time towards personal growth. Many of us believe that the doorway to success swings outward, and that the more out there and "in the world" we are, the more we'll achieve. However, the doorway to success also swings inward, meaning that personal mastery is just as essential, because it's linked to professional mastery. Next let's explore four aspects of our "interior' life, that will lead topersonal growth. The Four Interior Empires of Personal Growth You've probably heard of "mindset," but have you ever heard the terms: heartset, healthset, and soulset? According to Sharma, mindset matters when it comes to achieving our goals, but it isn't the only important facet of personal growth. Sharma says our performance often reflects our "story"; how we see ourselves and what we think we're capable of. Therefore, thinking optimistically, cultivating gratitude, using positive reframes, and having a go-getter attitude are essential traits that can strengthen our minds and help us reach our goals. However, in addition to cultivating a healthy mindset, three other interior empires need to be developed. These are heartset, healthset, and soulset. Let's briefly explore each. A great mindset can inspire a positive mental attitude, but if our heart (our emotional life) feels difficult, it'll be hard to work from a space of creative productivity. At times we all feel insecure or harbor difficult emotions such as anger, frustration and heartbreak. Hence, it's vital that we are receptive to these feelings, process them, and learn to let go of the emotions that no longer serve us. Working on our heartset can be achieved through meditation, journaling, spending more time in nature, or talking with a therapist. Our author believes that energy is even more valuable than intelligence. We could have a great product and a bustling business with an inspiring mission. Still, if we don't have the fire or the energy to execute ourambitions over a long period, nothing valuable will ever get done. So the third empire we need to build is healthset. We need to focus on achieving optimal physical health to ensure that we can develop our gifts over a long period. Sharma says, 'The secret to being legendary is focusing on longevity.' The last of the four empires is soulset. How can we make our soul come alive? How can we feel more connected to ourselves, others, and the world around us? Perhaps we can find ways to disconnect from our technology-driven world, spend time in nature, and re-connect to ourselves, our values, and what drives us to create in the first place. Sharma believes that working daily on what makes our soul come alive, can help us answer critical work-related questions. This will help us answer questions such as, 'How can our work be a "ministry of mastery" to benefit fellow human beings? Or, 'What's a cause that's larger than ourselves, and how do we live not for me, but for we. ' Now that we've explored why waking up at 5 am is worth our while, let's move towards some strategies to help us get out of bed, and start every day the right way. To Perform Like the Top 5%, We Need To Stop Acting Like the 95% When our morning alarm goes off, many of us might hit snooze, and roll over to get a few minutes or even hours of extra sleep. People who do this tend to rise later, and usually feel groggy as they rush to get to work on time. This might be the start of an average person's day; but is average the best you can do?Science shows that it takes sixty-six days to form a new neural pathway. So if you like hitting snooze, know that initially getting up pre-dawn is going to feel difficult and unnatural. You'll have to summon all your willpower. Expect to feel a little bit out of control, confused, and you'll want to quit. However, every single one of these feelings is normal, and each day you commit to this new habit is a step closer to making it stick. What follows are three key tips to rise and shine, and boost productivity at 5 am. Move, Reflect and Grow For those of us who like numbers, Sharma offers us the 20 20 20 formula. He breaks up the first hour into three twenty-minute slots, where we move, reflect, and review our goals to ensure that getting up at 5 am delivers results. So perhaps get the timer out and take note. From 5 am to 5:20 am: get up and break a sweat. Boost your mood and brainpower. When we sweat, we get rid of cortisol (the stress hormone), leaving us in a better, more balanced mood for the day ahead. Exercising to the point that makes us sweat, generates the protein BDNF (brainderived neurotrophic factor), which repairs brain cells and accelerates the formation of new neural connections. Moving our muscles literally sharpens our attention muscle. By the end of a twenty-minute sweat session, we're more psychologically sound and focused. So exercise for twenty minutes to make sure you're awake and focused, and perform any vigorous physical activity that gets you sweating. Go for a run, do some jumping jacks, or follow along to an exercise video. From 5:20 am to 5:40 am: dedicate time to reflect. Get out your journal,practice gratitude, plan your day, and try to meditate. Take hold of the tranquil time before the rest of the world wakes up and tries to distract you. Time for reflection is how we fortify all four internal empires. Meditation practice is also proven to lower cortisol, deepen awareness and cultivate a sense of presence. Tune into what serves you, let go of what doesn't, and connect to your driving vision. From 5:40 am to 6 am: dedicate time towards learning and growth. Review goals, read books, watch documentaries, or study online. The one thing high-achievers share is a love of learning. Now that the body is awake and focused, this is the chance to deepen knowledge in whatever is most meaningful and fascinating. So there it is, three steps to deepen our awareness, elevate performance, and make the most of this one precious hour. However, this doesn't work if we don't get a full night's rest. We need to remember that a good morning starts with a good evening. Get a Good Night's Sleep How we spend the last hour of the day is just as important as how we spend the first. The secret to sustaining the ritual of waking up at 5 am, is to be in bed and ready to sleep by 10 pm. Just like the move rest and reflect formula, here's how to spend your three hours before bedtime. From 7 to 8 pm, make sure to have your last meal and switch off from the digital world. Studies show that blue light from our TVs and other tech devices reduces melatonin production, which is the chemical our bodyproduces to induce sleep. From 8 to 9 pm, use this as quiet downtime. Perhaps for some, this means engaging meaningfully with family or doing more solo activities like reading, meditating, and taking a bath. The goal is to unwind and relax in preparation for a deep night's sleep. From 9 to 10 pm, get out your "sweat session" clothes for the next morning, and get ready for bed. Remember to keep the room cool, dark, and tech-free. End off by trying to practice a little gratitude. Balance Work With Rest Sharma worked with the athlete Usain Bolt, who told him, 'I rest 50% of the time to allow the training to take effect.' This reiterates the fact that growth doesn't just happen during performance phases, but also in recovery phases. Elite athletes exert themselves to the fullest, and then they recover. Successful business people and creatives work in the same way. They work in sprints of intense focus and energy, and then they pull back and recover. They repeat this cycle over and over again.Sharma refers to this as "The Twin Cycles of Elite Performance." We've got our high-performance cycle, where we're all in, and then we pull back and enter a deep recovery cycle. That's when we refuel, regenerate and replenish. Why do we do this? When any muscle stretches to its limit, it needs time to recover and prepare for more strenuous activity. If all wedo is focus our willpower and brainpower all day long, we damage our productivity because we deplete necessary resources. So when it comes to planning the week ahead, try to take at least one or two days off, and have a mini tech-free vacation. Time off means no work, just doing things that genuinely rejuvenate you. This could be quality time alone, or with friends and family. Spend time in nature, take long walks, write in a journal, and engage in whatever creative activities that enable a state of flow and energy renewal. So, remember to embrace your work-life as if you're a top athlete, because rest and recovery matter.
"""

print(len(text))

In [ ]:
# Get the final summary
final_summary = iterative_summarization(text, target_word_count=1000)
print(final_summary)

In [ ]:
import gc
del model
gc.collect()
print("Model has been removed from the device and memory is freed.")